In [3]:
import torch 
import torch.nn as nn 

In [4]:
'''
1.self attention is a mechanism in transformers that is used to compute more efficient input representations by allowing each position in a sequence to interact with and weigh
the importance of all other positions within the same sequence 
2. The "self" in self attention ?
    >ability to compute attention weights by relating different positions within a single input sequence.
    >assesses and learns the relationship and dependencies between various parts of the inputs itself such as word in a senctence or pixels in an image 
what is the main goal of self-attention ?
    >compute a context vector for each input element, that combines information from all other input elements. 
    >importance or contribution of each input element for computing is determined by attention weights
    >when computing attention weights are calculated with respect to input element, and all other inputs. 
context vector: 
context vector is an embedding that contains information about all other inputs elements. It can be interpreted as an enriched embedding vector
what is the use of context vector? 
    > to create enriched representation of each elements in an input sequence like a sentence by incorporation information from all other elements in the sequence
    >helps to understand realtionsship and relevance of words in a sentecne to each other. 
    >Later we will add trainable weights that help an LLM learng to consturct these context vectors so that they are relevant for the LLM to generate the next token
'''

'\n1.self attention is a mechanism in transformers that is used to compute more efficient input representations by allowing each position in a sequence to interact with and weigh\nthe importance of all other positions within the same sequence \n2. The "self" in self attention ?\n    >ability to compute attention weights by relating different positions within a single input sequence.\n    >assesses and learns the relationship and dependencies between various parts of the inputs itself such as word in a senctence or pixels in an image \nwhat is the main goal of self-attention ?\n    >compute a context vector for each input element, that combines information from all other input elements. \n    >importance or contribution of each input element for computing is determined by attention weights\n    >when computing attention weights are calculated with respect to input element, and all other inputs. \ncontext vector: \ncontext vector is an embedding that contains information about all other 

In [5]:
'''
Attention mechanism the text generating decoder part of the network can acesss all input token selectively this mean that some input tokens are more important than others for generating 
a given output token 
This means that some input toekns are more importaant than others for genrating a given output token .

self in self attention refers to the mechanism ability to compute attention weights by relating different positions within a single input sequence . 
Learns the relationship amd dependencies between various parts of the input itself, such as sequence to sequence model where the attention might be between an input sequence and 
output sequence
'''

'\nAttention mechanism the text generating decoder part of the network can acesss all input token selectively this mean that some input tokens are more important than others for generating \na given output token \nThis means that some input toekns are more importaant than others for genrating a given output token .\n\nself in self attention refers to the mechanism ability to compute attention weights by relating different positions within a single input sequence . \nLearns the relationship amd dependencies between various parts of the input itself, such as sequence to sequence model where the attention might be between an input sequence and \noutput sequence\n'

In [6]:
import torch 
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)


In [7]:
query = inputs[1] # query vector using the inputs first index 
attention = torch.empty(inputs.shape[0]) #initializing an empty torch object to store the attention weights
for i,x_i in enumerate(inputs):
    attention[i] = torch.dot(x_i,query) #calculating dot product to calculate attention weights
print(attention)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [8]:
# a dot product is essentially just a concise way of multiplying two vectors element-wise and then summing the products
res = 0
for idx,element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]
print(res)

tensor(0.9544)


Normalizing the vector 

In [9]:
attention_weights_tmp = attention / attention.sum()
print(attention_weights_tmp)
print("sum",attention_weights_tmp.sum())

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
sum tensor(1.0000)


In [10]:
attention_weights_2 = torch.softmax(attention,dim=0)
print(attention_weights_2)
print("sum",attention_weights_2.sum())

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum tensor(1.)


In [11]:
#now that we computed the normalized attention weights we are ready for the final step 
#calculating the context vector by multiplying the embedded input tokens, with the correspoding attention weights and then summing the resulting vectors 


In [12]:
query = inputs[1]
context_vector = torch.zeros(query.shape)
for idx,x_i in enumerate(inputs):
    print(x_i)
    context_vector += x_i * attention_weights_2[idx]
print(context_vector)


tensor([0.4300, 0.1500, 0.8900])
tensor([0.5500, 0.8700, 0.6600])
tensor([0.5700, 0.8500, 0.6400])
tensor([0.2200, 0.5800, 0.3300])
tensor([0.7700, 0.2500, 0.1000])
tensor([0.0500, 0.8000, 0.5500])
tensor([0.4419, 0.6515, 0.5683])


In [13]:
'''
1.Compute attention scores
2.Compute attention weights 
3.Compute context vectors
'''

'\n1.Compute attention scores\n2.Compute attention weights \n3.Compute context vectors\n'

In [14]:
attn_scores = torch.empty(6,6)
for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i,x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [16]:

attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [17]:
attn_weights = torch.softmax(attn_scores,dim =1 )
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [19]:
all_context_vector = attn_weights @ inputs

In [21]:
all_context_vector

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [35]:
x_2  = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [36]:
torch.manual_seed(123)
W_query = nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key = nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value = nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [38]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [40]:
keys = inputs @ W_key 
values = inputs@ W_value
print("keys.shape",keys.shape)
print("value.shape",values.shape)

keys.shape torch.Size([6, 2])
value.shape torch.Size([6, 2])


In [41]:
key_2 = keys[1]
attn_score_22 = query_2.dot(key_2)
print(attn_score_22)

tensor(1.8524)


In [42]:
attn_score_2 = query_2 @ keys.T 
print(attn_score_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [44]:
keys.shape

torch.Size([6, 2])

In [43]:
keys.shape[-1]

2

In [46]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_score_2 / d_k ** 0.5,dim=-1)
print(attn_weights_2)
print(attn_weights_2.shape)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
torch.Size([6])


In [47]:
context_vector_2 = attn_weights_2 @ values 
print(context_vector_2)

tensor([0.3061, 0.8210])


In [55]:
import torch.nn as nn 
class SelfAttention(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.d_out = d_out 
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))
        self.W_key  = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))
    def forward(self,x):
        keys = x @ self.W_key
        query = x @ self.W_query
        value = x @ self.W_value
        attn_scores = query @ keys.T 
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        context_vector = attn_weights @ value
        return context_vector

In [56]:
torch.manual_seed(123)
sa_v1 = SelfAttention(d_in,d_out)
print(sa_v1(inputs))

tensor([[0.1771, 0.4980],
        [0.1794, 0.5031],
        [0.1819, 0.5064]], grad_fn=<MmBackward0>)


In [61]:
import torch.nn as nn 
class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias= False):
        super().__init__()
        self.d_out = d_out 
        self.W_query = nn.Linear(d_in,d_out,bias = False)
        self.W_key  = nn.Linear(d_in,d_out,bias = False)
        self.W_value = nn.Linear(d_in,d_out,bias= False)
    def forward(self,x):
        keys =  self.W_key(x)
        query =  self.W_query(x)
        value =  self.W_value(x)
        attn_scores = query @ keys.T 
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        context_vector = attn_weights @ value
        return context_vector

In [63]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in,d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
